In [1]:
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/recurrent-neural-networks/char-rnn/data/anna.txt

--2019-07-11 14:23:45--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/recurrent-neural-networks/char-rnn/data/anna.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2025486 (1.9M) [text/plain]
Saving to: ‘anna.txt’

anna.txt            100%[===================>]   1.93M  --.-KB/s    in 0.04s   

2019-07-11 14:23:46 (47.6 MB/s) - ‘anna.txt’ saved [2025486/2025486]



In [0]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [6]:
encoded[:100]

array([48, 21, 75,  7, 49, 70, 80, 32, 76, 18, 18, 18, 81, 75,  7,  7, 20,
       32, 56, 75,  1, 35, 65, 35, 70, 77, 32, 75, 80, 70, 32, 75, 65, 65,
       32, 75, 65, 35,  4, 70, 30, 32, 70, 44, 70, 80, 20, 32, 43, 22, 21,
       75,  7,  7, 20, 32, 56, 75,  1, 35, 65, 20, 32, 35, 77, 32, 43, 22,
       21, 75,  7,  7, 20, 32, 35, 22, 32, 35, 49, 77, 32, 62, 66, 22, 18,
       66, 75, 20, 27, 18, 18, 58, 44, 70, 80, 20, 49, 21, 35, 22])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[48 21 75  7 49 70 80 32 76 18]
 [77 62 22 32 49 21 75 49 32 75]
 [70 22 79 32 62 80 32 75 32 56]
 [77 32 49 21 70 32 82 21 35 70]
 [32 77 75 66 32 21 70 80 32 49]
 [82 43 77 77 35 62 22 32 75 22]
 [32 46 22 22 75 32 21 75 79 32]
 [15  6 65 62 22 77  4 20 27 32]]

y
 [[21 75  7 49 70 80 32 76 18 18]
 [62 22 32 49 21 75 49 32 75 49]
 [22 79 32 62 80 32 75 32 56 62]
 [32 49 21 70 32 82 21 35 70 56]
 [77 75 66 32 21 70 80 32 49 70]
 [43 77 77 35 62 22 32 75 22 79]
 [46 22 22 75 32 21 75 79 32 77]
 [ 6 65 62 22 77  4 20 27 32 36]]


In [15]:
import torch
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
import torch.nn as nn

class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [19]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [20]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2548... Val Loss: 3.2188
Epoch: 1/20... Step: 20... Loss: 3.1394... Val Loss: 3.1341
Epoch: 1/20... Step: 30... Loss: 3.1405... Val Loss: 3.1235
Epoch: 1/20... Step: 40... Loss: 3.1106... Val Loss: 3.1185
Epoch: 1/20... Step: 50... Loss: 3.1465... Val Loss: 3.1173
Epoch: 1/20... Step: 60... Loss: 3.1206... Val Loss: 3.1148
Epoch: 1/20... Step: 70... Loss: 3.1052... Val Loss: 3.1128
Epoch: 1/20... Step: 80... Loss: 3.1226... Val Loss: 3.1075
Epoch: 1/20... Step: 90... Loss: 3.1178... Val Loss: 3.0996
Epoch: 1/20... Step: 100... Loss: 3.0947... Val Loss: 3.0838
Epoch: 1/20... Step: 110... Loss: 3.0592... Val Loss: 3.0362
Epoch: 1/20... Step: 120... Loss: 2.9677... Val Loss: 2.9555
Epoch: 1/20... Step: 130... Loss: 2.8449... Val Loss: 2.8064
Epoch: 2/20... Step: 140... Loss: 2.7484... Val Loss: 2.6867
Epoch: 2/20... Step: 150... Loss: 2.6262... Val Loss: 2.5775
Epoch: 2/20... Step: 160... Loss: 2.5539... Val Loss: 2.5173
Epoch: 2/20... Step: 170... Loss:

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [26]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, the crush there was the ministry
and heard, and was not at once another wife, this thought that was not a could not come in the man when he seemed, talking to all
her, and he would come to a minute.

He trinked the concentrating the sorry, the carriage was
stoping.

"And I don't beait my sere consider. We am has her from the meadow," then the part had been saying his bridegriem.

"In his franking table in
the same sign, a man would
be a servant of it," said Lizaveta Pecrovna, he supposed talking of the subject of his head.

At the same and with his strange silence in the sides as a matter
and all of the frond of a long walking of the change and sole on the simple that had a more of all at that his carriage. At the moment were the fact that the fourdie changes the courtes were
answer for that home too he seemed so here, took the point, but something had taken any
merries, and he was settling it with his
beautiful
figure. And, so such a musural came of them. He went up to
the child

In [27]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [29]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Shudipto said"))

And Shudipto said: "I don't know that I'm going
for the money
than in
that
sister-are three steps of
mysilf in all her tastes and such a conformance, and all about that man a child having to be so fancied the state of home. He was truthing the might humble, where they work and the children, and when the sale that he was not to
suppose the man strong at those answing to the
strength. All the possible stronger took
her to her frunt, and the
princess were never
from the sick men with herself too, was it, and was not any most influence to be all hos and horses and somewhat he felt
and anywhere, and all together this mistake the steps of the moment hust before, he went into her face. She wanted to speak to Levin, and how he saw it. The formed house as a shill was horrible to the
memory.

Stepan Arkadyevitch, which she sat several times, he would not come on
them with the same steps.

"You would have train to me to be so in the pispose
to me the sister of him. What is seeing her?"

And he wa